In [56]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from collections import Counter

API_KEY = '3102ef91f3e039d1d49f03cb0537acab'  # Replace with your actual API key
DISCORD_WEBHOOK_URL = 'https://discordapp.com/api/webhooks/1402749967172636795/ew4ANKYWlZy-aGC6FHPk406ZeVW4_Nc8sObcnwMY4sZmS9110u6svtKZqmhpYay0ycAP'


In [61]:
excluded_tickers = ['BF.B', 'BRK.B', 'ZBRA']

def get_sp500_tickers(excluded_tickers=None):
    if excluded_tickers is None:
        excluded_tickers = []
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url)[0]
    tickers = table['Symbol'].tolist()
    # Adjust for dot notation in FMP (e.g., BRK.B → BRK-B)
    tickers = [t.replace('.', '-') for t in tickers if t not in excluded_tickers]
    return tickers



def fetch_price_data_fmp_full(ticker, start_date, end_date):
    url = (
        f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}"
        f"?from={start_date.strftime('%Y-%m-%d')}&to={end_date.strftime('%Y-%m-%d')}"
        f"&apikey={API_KEY}"
    )
    
    r = requests.get(url)
    if r.status_code != 200:
        print(f"❌ Failed to fetch data for {ticker}: Status {r.status_code}")
        return None
    
    json_data = r.json()
    if 'historical' not in json_data:
        print(f"❌ No historical data found for {ticker}")
        return None
    
    df = pd.DataFrame(json_data['historical'])
    if df.empty:
        print(f"❌ Historical data is empty for {ticker}")
        return None
    
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    
    # Return the close prices with generic column name
    return df[['close']]

def send_to_discord(message):
    data = {"content": message}
    r = requests.post(DISCORD_WEBHOOK_URL, json=data)
    return r.status_code == 204  # 204 means success for webhook post

def send_long_message_to_discord(message, max_len=1900):
    import requests
    chunks = [message[i:i+max_len] for i in range(0, len(message), max_len)]
    for chunk in chunks:
        data = {"content": chunk}
        r = requests.post(DISCORD_WEBHOOK_URL, json=data)
        if r.status_code != 204:
            print("Failed to send message chunk.")
            return False
    return True

In [52]:
price_data = {}
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

sp500_tickers = get_sp500_tickers(excluded_tickers)

for ticker in sp500_tickers:

    print(f"Fetching {ticker}...")
    df = fetch_price_data_fmp_full(ticker, start_date, end_date)
    if df is not None:
        price_data[ticker] = df
    else:
        print(f"⚠️ No data for {ticker}")


Fetching MMM...
Fetching AOS...
Fetching ABT...
Fetching ABBV...
Fetching ACN...
Fetching ADBE...
Fetching AMD...
Fetching AES...
Fetching AFL...
Fetching A...
Fetching APD...
Fetching ABNB...
Fetching AKAM...
Fetching ALB...
Fetching ARE...
Fetching ALGN...
Fetching ALLE...
Fetching LNT...
Fetching ALL...
Fetching GOOGL...
Fetching GOOG...
Fetching MO...
Fetching AMZN...
Fetching AMCR...
Fetching AEE...
Fetching AEP...
Fetching AXP...
Fetching AIG...
Fetching AMT...
Fetching AWK...
Fetching AMP...
Fetching AME...
Fetching AMGN...
Fetching APH...
Fetching ADI...
Fetching AON...
Fetching APA...
Fetching APO...
Fetching AAPL...
Fetching AMAT...
Fetching APTV...
Fetching ACGL...
Fetching ADM...
Fetching ANET...
Fetching AJG...
Fetching AIZ...
Fetching T...
Fetching ATO...
Fetching ADSK...
Fetching ADP...
Fetching AZO...
Fetching AVB...
Fetching AVY...
Fetching AXON...
Fetching BKR...
Fetching BALL...
Fetching BAC...
Fetching BAX...
Fetching BDX...
Fetching BBY...
Fetching TECH...
Fetching

In [53]:
import pandas as pd
from datetime import timedelta
from collections import Counter

def backtest_momentum_strategy_stoploss(start_date, end_date, price_data, top_n=2, excluded_tickers=None, stop_loss_pct=0.10):
    data = pd.concat(price_data, axis=1).dropna(axis=1)
    data.index = data.index.tz_localize(None)

    trade_log = []
    monthly_returns = []
    top_long_tickers = []
    bottom_short_tickers = []

    max_months = (end_date - start_date).days // 30

    for i in range(1, max_months):  # start from month 1
        lookback_start = start_date + timedelta(days=(i - 1) * 30)
        lookback_end = lookback_start + timedelta(days=30)

        entry_date = lookback_end
        exit_date = entry_date + timedelta(days=30)

        lookback_data = data.loc[lookback_start:lookback_end]
        trade_data = data.loc[entry_date:exit_date]

        if lookback_data.empty or trade_data.empty or len(trade_data) < 2:
            continue

        # Rank tickers using prior month returns
        lookback_returns = (lookback_data.iloc[-1] - lookback_data.iloc[0]) / lookback_data.iloc[0]

        if excluded_tickers:
            lookback_returns = lookback_returns.drop(labels=excluded_tickers, errors='ignore')

        top_performers = lookback_returns.nlargest(top_n).index.tolist()
        worst_performers = lookback_returns.nsmallest(top_n).index.tolist()

        top_long_tickers.extend(top_performers)
        bottom_short_tickers.extend(worst_performers)

        # Entry price on first day of month
        entry_prices = trade_data.iloc[0]
        # Used to track daily P&L
        portfolio_returns = []

        # Loop through each day to simulate early stop-loss exit
        for day_index in range(1, len(trade_data)):
            daily_prices = trade_data.iloc[day_index]
            daily_returns_long = (daily_prices[top_performers] - entry_prices[top_performers]) / entry_prices[top_performers]
            daily_returns_short = (entry_prices[worst_performers] - daily_prices[worst_performers]) / entry_prices[worst_performers]
            daily_portfolio_return = daily_returns_long.mean() + daily_returns_short.mean()
            portfolio_returns.append(daily_portfolio_return)

            if daily_portfolio_return <= -stop_loss_pct:
                # Stop loss hit, exit on this day
                actual_exit_date = trade_data.index[day_index]
                exit_prices = daily_prices

                long_returns = daily_returns_long
                short_returns = daily_returns_short
                stop_triggered = True
                break
        else:
            # No stop loss hit — exit at end of month
            actual_exit_date = trade_data.index[-1]
            exit_prices = trade_data.iloc[-1]

            long_returns = (exit_prices[top_performers] - entry_prices[top_performers]) / entry_prices[top_performers]
            short_returns = (entry_prices[worst_performers] - exit_prices[worst_performers]) / entry_prices[worst_performers]
            stop_triggered = False

        # Save overall portfolio return
        monthly_return = long_returns.mean() + short_returns.mean()
        monthly_returns.append(monthly_return)

        # Log trades
        for ticker in top_performers:
            trade_log.append({
                'type': 'LONG',
                'ticker': ticker,
                'entry_date': entry_date.date(),
                'exit_date': actual_exit_date.date(),
                'entry_price': float(entry_prices[ticker]),
                'exit_price': float(exit_prices[ticker]),
                'return': float(long_returns[ticker]),
                'stop_loss_hit': stop_triggered
            })

        for ticker in worst_performers:
            trade_log.append({
                'type': 'SHORT',
                'ticker': ticker,
                'entry_date': entry_date.date(),
                'exit_date': actual_exit_date.date(),
                'entry_price': float(entry_prices[ticker]),
                'exit_price': float(exit_prices[ticker]),
                'return': float(short_returns[ticker]),
                'stop_loss_hit': stop_triggered
            })

    # Calculate total return
    portfolio_return = (1 + pd.Series(monthly_returns)).prod() - 1

    top_5_longs = [ticker for ticker, _ in Counter(top_long_tickers).most_common(5)]
    top_5_shorts = [ticker for ticker, _ in Counter(bottom_short_tickers).most_common(5)]

    # Suggest next tickers to trade based on last complete month
    recent_start = end_date - timedelta(days=30)
    recent_data = data.loc[recent_start:end_date]

    if not recent_data.empty and len(recent_data) >= 2:
        recent_returns = (recent_data.iloc[-1] - recent_data.iloc[0]) / recent_data.iloc[0]

        if excluded_tickers:
            recent_returns = recent_returns.drop(labels=excluded_tickers, errors='ignore')

        stocks_to_buy_now = recent_returns.nlargest(top_n).items()
        stocks_to_short_now = recent_returns.nsmallest(top_n).items()
    else:
        stocks_to_buy_now = []
        stocks_to_short_now = []

    return (
        trade_log,
        monthly_returns,
        portfolio_return,
        top_5_longs,
        top_5_shorts,
        stocks_to_buy_now,
        stocks_to_short_now
    )


In [71]:
def main():
    start_date = pd.Timestamp("2024-08-01")
    end_date = pd.Timestamp("2025-08-01")
    #end_date = pd.Timestamp(datetime.today().date())  # Today’s date (no time part)
    #start_date = end_date - pd.Timedelta(days=365)
    excluded_tickers = None

    (
        trade_log,
        monthly_returns,
        portfolio_return,
        top_5_longs,
        top_5_shorts,
        stocks_to_buy_now,
        stocks_to_short_now
    ) = backtest_momentum_strategy_stoploss(
        start_date, end_date, price_data, top_n=2, excluded_tickers=excluded_tickers, stop_loss_pct=0.10
    )

    message_lines = []
    message_lines.append("\n💰 **Total Portfolio Return:**")
    message_lines.append(f"{portfolio_return:.2%}")
    print("📈 Monthly Performance:")
    for i, monthly_return in enumerate(monthly_returns, 1):
        print(f"Month {i}: {monthly_return:.2%}")

    print("\n📄 Trade Log:")
    for trade in trade_log:
        sl_text = " [STOP LOSS HIT]" if trade['stop_loss_hit'] else ""
        print(f"{trade['entry_date']} → {trade['exit_date']} | {trade['type']} {trade['ticker']} | "
              f"Entry: {trade['entry_price']:.2f}, Exit: {trade['exit_price']:.2f}, "
              f"Return: {trade['return']:.2%}{sl_text}")

    print("\n💰 Total Portfolio Return:")
    print(f"{portfolio_return:.2%}")

    print("\n📌 Stocks to BUY NOW (based on last 30 days):")
    message_lines.append("\n📌 **Stocks to BUY NOW (based on last 30 days):**")
    for ticker, ret in stocks_to_buy_now:
        print(f"{ticker[0]}: {ret:.2%}")  # Access only the ticker symbol (first element)
        message_lines.append(f"{ticker[0]}: {ret:.2%}")
    
    print("\n📌 Stocks to SHORT NOW (based on last 30 days):")
    message_lines.append("\n📌 **Stocks to SHORT NOW (based on last 30 days):**")
    for ticker, ret in stocks_to_short_now:
        print(f"{ticker[0]}: {ret:.2%}")  # Access only the ticker symbol (first element)
        message_lines.append(f"{ticker[0]}: {ret:.2%}")


    # Join everything into one message
    message = "\n".join(message_lines)

    # Send message to Discord
    success = send_long_message_to_discord(message)
    if success:
        print("Message sent to Discord!")
    else:
        print("Failed to send message to Discord.")

if __name__ == "__main__":
    main()


📈 Monthly Performance:
Month 1: 4.99%
Month 2: 7.17%
Month 3: 36.76%
Month 4: 5.60%
Month 5: -13.14%
Month 6: 5.53%
Month 7: 10.69%
Month 8: -1.23%
Month 9: 27.13%
Month 10: 6.65%
Month 11: 5.56%

📄 Trade Log:
2024-08-31 → 2024-09-30 | LONG ('FTNT', 'close') | Entry: 77.13, Exit: 77.55, Return: 0.54%
2024-08-31 → 2024-09-30 | LONG ('SBUX', 'close') | Entry: 93.18, Exit: 97.49, Return: 4.63%
2024-08-31 → 2024-09-30 | SHORT ('DG', 'close') | Entry: 83.79, Exit: 84.57, Return: -0.93%
2024-08-31 → 2024-09-30 | SHORT ('SMCI', 'close') | Entry: 44.18, Exit: 41.64, Return: 5.75%
2024-09-30 → 2024-10-30 | LONG ('VST', 'close') | Entry: 118.54, Exit: 124.11, Return: 4.70%
2024-09-30 → 2024-10-30 | LONG ('CEG', 'close') | Entry: 260.02, Exit: 261.78, Return: 0.68%
2024-09-30 → 2024-10-30 | SHORT ('DLTR', 'close') | Entry: 70.32, Exit: 63.31, Return: 9.97%
2024-09-30 → 2024-10-30 | SHORT ('MCK', 'close') | Entry: 494.42, Exit: 499.34, Return: -1.00%
2024-10-30 → 2024-11-29 | LONG ('UAL', 'close')